* DATASET: Apresentar dados que serão utilizados na soluçao.

* ANÁLISE: Planejar e gerar as análises que serão realizadas com os dados propostos em código python.

* INSIGHT: Apresentar resultados (em forma de listagem e/ou gráfico) e gerar insights relacionados a solução proposta. Os insights deverão ser disponibilizados como comentários no arquivo, logo após o resultado. Considere que os insights precisam ser bem compreendidos. Portanto, explique bem seus insights.
 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
df_ota_bus_ticket_sales = spark.table("bus.bronze.ota_bus_ticket_sales")

df_ota_bus_ticket_sales = df_ota_bus_ticket_sales.withColumn("year_purchase", year(col("date_purchase"))) \
                   .withColumn("month_purchase", month(col("date_purchase"))) \
                   .withColumn("day_of_week_purchase", dayofweek(col("date_purchase")))

In [0]:
print("Informações gerais:")
df_ota_bus_ticket_sales.printSchema()

### Estatísticas Descritivas
- GMV médio: R$ 275 (faixa entre R$ 52 e R$ 500)  
- Tickets por compra: média de 3 (máx: 5)  
- Período de dados: 2020–2025, foco em 2022–2023

In [0]:
print("\nEstatísticas descritivas:")
display(df_ota_bus_ticket_sales.describe())

### Distribuição dos Valores de Compras (GMV)
-  Compras variam entre R$ 50 e R$ 500  
-  Distribuição uniforme não há concentração em faixas específicas  
-  Não existem outliers extremos, garantindo consistência na base  


In [0]:
df_pandas = df_ota_bus_ticket_sales.select("gmv_success").toPandas()
plt.figure(figsize=(10, 5))
sns.histplot(df_pandas["gmv_success"])
plt.xlabel("Valor das compras R$")
plt.ylabel("Frequência de compras")
plt.title("Distribuição dos valores de compras")
display(plt.gcf())
plt.clf()

### Distribuição dos Valores Médios por Passagem
- Maioria das passagens custa entre R$ 70 e R$ 120
- Forte concentração em valores baixos  passagens populares e curtas  
- Existe uma cauda longa até R$ 500, mas são casos menos frequentes  


In [0]:
# converti para pandas com GMV e quantidade de tickets
df_pandas = df_ota_bus_ticket_sales.select(
    "gmv_success", "total_tickets_quantity_success"
).toPandas()

# creie uma nova coluna para o valor médio das passagens
df_pandas["valor_por_ticket"] = (
    df_pandas["gmv_success"] / df_pandas["total_tickets_quantity_success"]
)

# plot do histograma usando valor médio por ticket
plt.figure(figsize=(10, 5))
sns.histplot(df_pandas["valor_por_ticket"], bins=40, kde=False)

plt.xlabel("Valor médio por passagem (R$)")
plt.ylabel("Frequência de tickets")
plt.title("Distribuição dos valores médios por passagem")
display(plt.gcf())
plt.clf()


### Valor Médio por Passagem (por cliente)
-  A maior parte dos clientes paga entre **R$ 70 e R$ 120** por passagem.   
-  Poucos clientes ultrapassam **R$ 150**, indicando que passagens mais caras são exceção.  
-  Isso reforça que o mercado é dominado por bilhetes de baixo a médio valor.  


In [0]:
from pyspark.sql.functions import col

# Agrupar por cliente (CPF ou ID do contato)
clientes_1 = df_ota_bus_ticket_sales.groupby("fk_contact").agg({
    "gmv_success": "sum",                    
    "total_tickets_quantity_success": "sum" 
})

# Criar a métrica: valor gasto médio por ticket
clientes_1 = clientes_1.withColumn(
    "valor_por_ticket",
    col("sum(gmv_success)") / col("sum(total_tickets_quantity_success)")
)

# Visualizar amostra
display(clientes_1)

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns

pdf = clientes_1.select("valor_por_ticket").toPandas()

plt.figure(figsize=(10,5))
sns.histplot(pdf["valor_por_ticket"], bins=40, kde=False)
plt.xlabel("Valor médio por passagem (R$)")
plt.ylabel("Qtd de clientes")
plt.title("Distribuição do valor médio por passagem (por cliente)")
plt.tight_layout()
display(plt.gcf()); plt.clf()


### Top Destinos Mais Procurados
-  Destinos com maior demanda: **Cuiabá, Teresina, Goiânia, Brasília e Rio de Janeiro**  
-  Forte presença de **capitais regionais** além das grandes metrópoles (SP, RJ)  
-  Indica que a procura não está centralizada só em grandes hubs, mas espalhada pelo Brasil  


In [0]:
top_destinos = df_ota_bus_ticket_sales.groupBy("place_destination_departure").count().orderBy(col("count").desc())
top_destinos_pandas = top_destinos.toPandas()
plt.figure(figsize=(10, 5))
sns.barplot(x=top_destinos_pandas["count"], y=top_destinos_pandas["place_destination_departure"])
plt.title("Top destinos mais procurados")
display(plt.gcf())
plt.clf()

### GMV Total por Cidade (Destinos Mais Rentáveis)
- **Destinos líderes em faturamento:** Salvador, Cuiabá, Goiânia, Porto Alegre e Brasília  
-  Esses destinos se destacam não apenas pelo volume, mas também pelo **valor agregado das passagens**  
-  Nem sempre os destinos mais procurados são os mais rentáveis. Aqui vemos equilíbrio entre demanda e ticket médio  


In [0]:
from pyspark.sql import functions as F

# GMV total por cidade
gmv_por_cidade = (
    df_ota_bus_ticket_sales
    .groupBy("place_destination_departure")
    .agg(F.sum("gmv_success").alias("gmv_total"))
    .orderBy(F.col("gmv_total").desc()) 
)
display(gmv_por_cidade)

Databricks visualization. Run in Databricks to view.

### Top Destinos Mais Procurados (por Tickets)
-  Cuiabá, Teresina, Rio de Janeiro e Brasília lideram em volume de passagens emitidas  
-  Há forte presença de **capitais regionais**, não apenas grandes metrópoles  
-  Isso sugere que a demanda não está concentrada apenas nos grandes hubs (SP e RJ), mas bem distribuída em cidades médias e destinos turísticos  


In [0]:
# Agregar destinos com contagem de compras e tickets
agg_destinos = (
    df_ota_bus_ticket_sales
    .groupBy("place_destination_departure")
    .agg(
        F.count("*").alias("compras_total"),
        F.sum("total_tickets_quantity_success").alias("tickets_total"),
        F.sum("gmv_success").alias("gmv_total")
    )
)

# Top mais procurados
mais_procurados = agg_destinos.orderBy(F.col("tickets_total").desc())

# Top menos procurados
menos_procurados = agg_destinos.orderBy(F.col("tickets_total").asc())


In [0]:
pdf_mais = mais_procurados.toPandas()
pdf_menos = menos_procurados.toPandas()

# --- Mais procurados
plt.figure(figsize=(10,6))
sns.barplot(data=pdf_mais, x="tickets_total", y="place_destination_departure", palette="Blues_d")
plt.title("Top Procuras (por tickets)")
plt.xlabel("Total de tickets")
plt.ylabel("Destino")
plt.tight_layout()
display(plt.gcf())
plt.clf()

In [0]:
%sql
select
  date_purchase,
  place_destination_departure,
  count(distinct nk_ota_localizer_id) as qtd,
  sum(gmv_success) as gmv_success
from
  bus.gold.ota_bus_ticket_sales
group by
  all
order by
  qtd desc;


Databricks visualization. Run in Databricks to view.

### Quantidade de compra por cliente
- Esse resultado mostra clientes de maior recorrência em destinos específicos. Alguns clientes compram várias vezes para a mesma cidade, representando um padrão de lealdade de viagem. Isso é estratégico para segmentação de marketing e campanhas de retenção.

In [0]:
%sql
select
  fk_contact,
  place_destination_departure,
  count(distinct nk_ota_localizer_id) as qtd,
  sum(gmv_success) as gmv_success
from
  bus.gold.ota_bus_ticket_sales
group by
  all
order by
  qtd desc;

## 📊 Análise dos Clusters de Clientes (Frequência x Gasto)

### Resumo dos Perfis Identificados

| Cluster | Perfil de Cliente | Características | Estratégia Recomendada |
|---------|------------------|-----------------|------------------------|
| 🔵 **Cluster 0** | Frequentes de Baixo/Médio Valor | ~7 compras em média, gasto total ~R$ 360, recência alta (~200 dias) | Monitorar churn → campanhas de reativação com descontos e ofertas simples |
| 🟠 **Cluster 1** | Premium (Alta Frequência e Gasto) | ~13 compras, gasto médio mais alto (~R$ 470), recência baixa (~128 dias) | Fidelização → programas VIP, benefícios exclusivos e cross-sell |
| 🟢 **Cluster 2** | Ex-Bons Clientes / Inativos | ~8 compras, gasto médio ~R$ 456, recência muito alta (~583 dias) | Recuperação → campanhas segmentadas, e-mails de “volte a viajar” |
| 🔴 **Cluster 3** | Intermediários Ativos | ~8 compras, gasto médio ~R$ 467, recência baixa (~136 dias) | Estímulo de Crescimento → promoções personalizadas para aumentar frequência e ticket médio |

---

### ✅ Insight Final
- O **Cluster 1 (Premium)** é o grupo mais valioso → prioridade para retenção.  
- O **Cluster 2 (Inativos)** traz oportunidade de reativação → clientes já provaram valor.  
- O **Cluster 0 (Frequentes de baixo gasto)** garantem volume, mas não receita → foco em upsell.  
- O **Cluster 3 (Intermediários)** ainda ativos → grande potencial de evolução com incentivos.  


In [0]:
from pyspark.sql.functions import max, count, col, lit, datediff

clientes = df_ota_bus_ticket_sales.groupBy("fk_contact").agg(
    max("gmv_success").alias("total_gasto"),
    count("nk_ota_localizer_id").alias("qtd_compras"),
    max("date_purchase").alias("ultima_compra")
)

max_date = df_ota_bus_ticket_sales.select(
    max("date_purchase")
).collect()[0][0]

clientes = clientes.withColumn(
    "recencia",
    datediff(
        lit(max_date),
        col("ultima_compra")
    )
)

rfm = clientes.select(
    "recencia",
    "qtd_compras",
    "total_gasto"
).toPandas()

rfm_norm = (rfm - rfm.min()) / (rfm.max() - rfm.min())

In [0]:
kmeans = KMeans(n_clusters=4, random_state=42)
rfm["cluster"] = kmeans.fit_predict(rfm_norm)

plt.figure(figsize=(10, 5))
sns.scatterplot(data=rfm, x="qtd_compras", y="total_gasto", hue="cluster", palette="tab10")

plt.title("Clusterização de Clientes (Frequência x Gasto)")
display(plt.gcf())
plt.clf()

rfm["fk_contact"] = clientes.select("fk_contact").toPandas()
print("\nResumo dos clusters:")
rfm_grouped = rfm.groupby("cluster").agg({
    "qtd_compras": "mean",
    "total_gasto": "mean",
    "recencia": "mean",
    "fk_contact": "count"
})
display(rfm_grouped)